# 1. 이터레이터란?
데이터를 하나씩 순서대로 꺼내올 수 있는 객체

In [3]:
# 리스트는 이터레이터일까?

a = [1, 2, 3]
next(a)

TypeError: 'list' object is not an iterator

In [21]:
# iter 함수를 이용해 이터레이터로 변환

a = [1, 2, 3]
ia = iter(a)
type(ia)

list_iterator

In [22]:
# 변경된 이터레이터 객체를 next 함수로 호출
# 더는 반환할 값이 없다면 StopIteration 예외 발생

next(ia)

1

In [23]:
next(ia)

2

In [24]:
next(ia)

3

In [25]:
next(ia)

StopIteration: 

In [7]:
# 이터레이터의 값을 가져오는 가장 일반적인 방법(for문)

a = [1, 2, 3]
ia = iter(a)

for i in ia :
    print(i)

1
2
3


In [10]:
# next로 그 값을 한 번 읽으면 다시는 읽을 수 없다

for i in ia :
    print(i)

## 2. 이레이터 만들기

### 이레이터 클래스를 만드는 핵심 규칙
#### 1. __iter__ 메서드 : 이레이터 객체 자신을 반환
#### 2. __next__ 메서드 : 다음 값을 리턴, 리턴할 값이 없으면 StopIteration 예외 발생

### MyIterator 클래스의 동작 원리

#### 1. __init__ 메서드: 이터레이터를 초기화
##### - self.data:​ 반복할 데이터를 저장
##### - self.position:​ 현재 위치를 추적하는 변수 (0부터 시작)

#### 2. __iter__ 메서드: 이터레이터 객체 자신을 반환
##### - 이 메서드가 있어야 파이썬이 해당 객체를 반복 가능한 객체로 인식
##### - for 문, iter() 함수, next() 함수 등에서 사용하기 위해 필수
##### - 보통 return self로 자기 자신을 반환

#### 3. __next__ 메서드: 다음 값을 반환
##### - self.position을 이용해 현재 위치의 값을 가져온다
##### - 위치를 하나씩 증가시킴킴
##### - 더 이상 값이 없으면 StopIteration 예외를 발생시킴

In [11]:
# interator.py

class MyIterator :
    def __init__(self, data) :
        self.data = data
        self.position = 0

    def __iter__(self) :
        return self

    def __next__(self) :
        if self.position >= len(self.data) :
            raise StopIteration
        result = self.data[self.position]
        self.position += 1
        return result
    
if __name__ == "__main__" :
    i = MyIterator([1, 2, 3])
    for item in i :
        print(item)

1
2
3


In [12]:
# reviterator.py
# 입력받은 데이터를 역순으로 출력

class ReverseIterator :
    def __init__(self, data) :
        self.data = data
        self.position = len(self.data) - 1

    def __iter__(self) :
        return self

    def __next__(self) :
        if self.position < 0 :
            raise StopIteration
        result = self.data[self.position]
        self.position -= 1
        return result
    
if __name__ == "__main__" :
    i = ReverseIterator([1, 2, 3])
    for item in i :
        print(item)

3
2
1


## 3. 제너레이터란?
간단하게 이터레이터를 만드는 특별한 함수

### 제너레이터의 핵심 특징
#### - 일반 함수와 비슷하지만 return 대신 yield 키워드를 사용한다.
#### - yield를 만나면 값을 반환하고 함수 실행을 일시정지한다.
#### - 다시 호출하면 일시정지했던 지점부터 계속 실행한다.
#### - 음악 플레이어의 재생/정지 기능처럼 동작한다

In [14]:
# 가장 간단한 제너레이터의 예

def mygen() :
    yield 'a'
    yield 'b'
    yield 'c'

g = mygen()

print(f"type(g) : {type(g)}")

type(g) : <class 'generator'>


In [15]:
next(g)

'a'

In [16]:
next(g)

'b'

In [17]:
next(g)

'c'

In [18]:
next(g)

StopIteration: 

## 4. 제너레이터 표현식

In [26]:
# 1부터 1,000까지 각각의 숫자를 제곱한 값을 순서대로 반환

# generator.py

def mygen() :
    for i in range(1, 1000) :
        result = i * i
        yield result

gen = mygen()

print(next(gen))
print(next(gen))
print(next(gen))

1
4
9


In [29]:
# 제너레이터 표현식
# 리스트 컴프리헨션과 비슷함
# 리스트 컴프리헨션 = 리스트 / 제너레이터 표현식 = 튜플
gen = (i * i for i in range(1, 1000))

print(next(gen))
print(next(gen))
print(next(gen))

1
4
9


## 5. 제너레이터와 이터레이터

In [32]:
# 제너레이터를 이터레이터 클래스로 구현한 예

class MyIterator :
    def __init__(self) :
        self.data = 1

    def __iter__(self) :
        return self

    def __next__(self) :
        result = self.data * self.data
        self.data += 1
        if self.data >= 1000 :
            raise StopIteration
        return result

iter = MyIterator()
for i in range(5) :
    print(next(iter))

1
4
9
16
25


## 6. 제너레이터 활용하기
상황 : 시간이 오래 걸리는 작업을 처리해야 하는데, 모든 결과가 필요하지 않은 경우

In [34]:
# 일반적인 리스트 컴프리헨션을 사용한 예

# <이 코드의 문제점 >
# longtime_job() 함수는 1초씩 걸린다.
# 리스트를 만들 때 5번의 함수를 모두 미리 실행한다.
# 첫 번재 결과만 필요한데도 5초를 기다려야 한다.

# generator2.py
import time

def longtime_job() :
    print("job start")
    time.sleep(1) # 1초 지연 - 실제로는 DB 조회, 파일 처리 등을 시뮬레이션
    return "done"

# 리스트 컴프리헨션 : 5번의 작업을 모두 실행해서 리스트로 만든다
list_job = [longtime_job() for i in range(5)]
print(list_job[0]) # 첫 번째 결과만 필요한 상황

job start
job start
job start
job start
job start
done


In [35]:
# 제너레이터를 사용한 해결책

# generator2.py
import time

def longtime_jop() :
    print("job start")
    time.sleep(1)
    return "done"

# 제너레이터 표현식 : 함수를 미리 실행하지 않고 필요할 때만 실행
list_job = (longtime_jop() for i in range(5))
print(next(list_job)) # 첫 번째 값만 요청

job start
done


### 핵심 차이점
#### - [](리스트 컴프리헨션) <-> ()(제너레이터 표현식)
#### - list_job[0] <-> next(list_job)으로 값을 가져옴

### 제너레이터의 동작 방식
#### 1. (longtime_job() for i in range(5))는 함수를 즉시 실행하지 않는다
#### 2. next(list_job)을 호출할 때 비로소 첫 번째 longtime_job()만 실행한다.
#### 3. 나머지 4개는 실행되지 않는다.